In [4]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Divide-and-Conquer & Contraction


<!-- MCSS: Maximum Contiguous Subsequence Sum -->


## Maximum Continguous Subsequence Sum

<p>Given a sequence of integers, the&#160; <strong><em>Maximum Contiguous Subsequence Sum Problem</em></strong> (<span class="sans-serif">MCSS</span>) requires finding the contiguous subsequence of the sequence with maximum total sum:</p><br>
    $$\textsf{MCS}{}\,(a) = {\arg\max}_{0 \leq i,j < |a|} \left( {\left( \sum_{k=i}^j a[k]  \right)} \right).  $$ 
    
We define the sum of an empty sequence to $-\infty$.

MCSS is useful for many applications, from genomics to data science and finance.






<span style="color:red">**Example**</span>: For $a = \langle 1, -2, 0, 3, -1, 0, 2, -3 \rangle$ a maximum contiguous subsequence (MCSS) is $\langle\, 3, -1, 0, 2 \rangle$. Another is $\langle 0, 3, -1, 0, 2 \rangle.$</p>

<p align="center">
  <img width=40% src="figures\mcss_emp.png"/>
</p>


<br><br><br><br><br>
This is similar to Problem 3 on HW1 (`longest_run`). Let's now look at a **brute-force** approach to this problem. 

What is the solution space, and how long does it take to evaluate it?





We must consider every contiguous subsequence and evaluate the maximum element within each. There are $O(n^2)$ contiguous subsequences. 

```python

# nested tabulate
a = [1,2,3,4,5]
flatten(
    tabulate(lambda i: 
             tabulate(lambda j: a[i:i+j+1],
                      len(a)-i),
         len(a))
)

```

<br>
The brute-force approach takes $O(n^3)$ work and $O(\log n)$ span.

### How about iterative, sequential version?

```python

a = [1, -2, 0, 3, -1, 0, 2, -3]

```

In [1]:
## Kadane’s Algorithm - O(n) Solution

a = [1, -2, 0, 3, -1, 0, 2, -3]

print(a)

def mcss_iterate(a):
    # init
    max_cur = 0
    max_new = 0
    # iter
    for i in range(len(a)):
        max_new = max(max_new + a[i], 0)
        max_cur = max(max_cur, max_new)
    return max_cur


print(mcss_iterate(a))

[1, -2, 0, 3, -1, 0, 2, -3]
4


### Can we do better using divide-and-conquer?

As usual let's start by dividing the input into two equal parts and recursively finding the solution. If the MCSS is within either part entirely, then in the combine step we just need to return the subsequence with larger maximum. 

But what if the MCSS spans the two halves?

<img width=70% src="figures\mcss_combine.jpg"/>



<!-- ![mcss_combine.jpg](mcss_combine.jpg)
 -->
Example: $a = \langle 1, -2, 0, 3, -1, 0, 2, -3 \rangle$

- Split into left = $\langle 1, -2, 0, 3 \rangle$, right = $\langle -1, 0, 2, -3 \rangle$

- Left MCSS is $\langle 0, 3 \rangle$ (or just $\langle 3 \rangle$) $= 3$

- Right MCSS is $\langle 0, 2 \rangle$ $= 2$ 

- But, the best MCSS crossing the cut is $\langle 0, 3, -1, 0, 2 \rangle$ $=4$

<br><br>

> The maximum sum spanning the cut is the sum of the largest suffix on the left plus the largest prefix on the right.

<br><br>

Suppose we could identify an MCSS ending at position $\lfloor n/2 \rfloor$ and an MCSS beginning at position $\lfloor n/2 \rfloor$. Then we could add values of these to obtain a candidate MCSS for the whole sequence. Then the best of the three candidate solutions is an MCSS for the entire sequence.

- **MCSSS**$(i)$: maximum contiguous sum **starting** at position $i$
- **MCSSE**$(i)$: maximum contiguous sum **ending** at position $i$

Suppose MCSSE and MCSSS can be solved in $O(n)$ work and $O(\log n)$ span, and $\mathit{bestAcross}~(b, c)$ constructs an MCSS crossing the split using these solutions. Then we could give this divide-and-conquer algorithm.

<p><span class="math display">\[\begin{array}{l}
\mathit{MCSSDC}~a =  
\\  
~~~~\texttt{if}~ |a| = 0~\texttt{then}  
\\  
~~~~~~~~{-\infty}{}  
\\  
~~~~\texttt{else if}~|a| = 1 ~\texttt{then}  
\\   
~~~~~~~~a[0]  
\\  
~~~~\texttt{else}  
\\   
~~~~~~~~\texttt{let}  
\\   
~~~~~~~~~~~~(b, c)  = \mathit{splitMid}~a  
\\   
~~~~~~~~~~~~(m_b, m_c) = \left( \mathit{MCSSDC}~b \ ||\ \mathit{MCSSDC}~c \right)  
\\   
~~~~~~~~~~~~m_{bc} = \mathit{bestAcross}~(b, c)  
\\   
~~~~~~~~\texttt{in}  
\\   
~~~~~~~~~~~~\max\{m_b, m_c, m_{bc}\}  
\\   
~~~~~~~~\texttt{end}  
\end{array}\]</span></p>



**Correctness**:

We can proceed by induction as usual. The base case produces the correct results. For the induction step, we make the hypothesis that the recursively computed MCSS's for $b$ and $c$ are correct. With a correct implementation of $\mathit{bestAcross}$, we can conclude $\max\{m_b, m_c, m_{bc}\}$ is an MCSS.

**Work/Span**:

Using our assumption about $\mathit{bestAcross}$ we have that:

$$ W(n) = 2W(n/2) + O(n)$$

and

$$ S(n) = S(n/2) + O(\log n)$$

These yield $O(n\log n)$ work and $O(\log^2 n)$ span. What is a lower bound for the work? (It turns out we can [match this lower bound](https://www.diderot.one/courses/43/books/185/part/332/chapter/2684#segment-192144).)



But how do we obtain an MCSS starting at a specified position (MCSSS) or ending at a specified position (MCSSE)?

Because one end of the contiguous subsequence is fixed, it turns out that we can use `reduce`,  `scan` and `scanI` (an inclusive version of scan) to design algorithms for [MCSSS](https://www.diderot.one/courses/43/books/185/part/332/chapter/2684#atom-176120) and [MCSSE](https://www.diderot.one/courses/43/books/185/part/332/chapter/2684#atom-176121) using $O(n)$ work and $O(\log n)$ span for our application in which the start and end position is the middle of the input list. 




**MCSS starting at position $i$:**


The intuition for this approach is just that if we lock position $i$ for the subsequence, then computing a prefix sum and then subsequent maximum suffices.


<img width=70% src="figures\mcsss_fig.jpg"/>

<!-- ![mcsss_fig.jpg](mcsss_fig.jpg) -->


### Sparc Code

<p><span class="math display">\begin{array}{l}  
\mathit{MCSSSOpt}~a~i =  
\\   
~~~~\texttt{let}   
\\   
~~~~~~~~b = \mathit{scanI}~\text{'}\,{\texttt{+}}\,\text{'}~0~a~[i \cdots (|a|-1)]  
\\   
~~~~\texttt{in}  
\\   
~~~~~~~~\mathit{reduce}~\mathit{max}~{-\infty}{}~b  
\\   
~~~~\texttt{end}  
\end{array}</span></p>



### Python Code

left = $\langle 1, -2, 0, 3 \rangle$, right = $\langle -1, 0, 2, -3 \rangle$

```python
>>> scan(add, 0, [-1,0,2,-3])
([-1, -1, 1, -2], -2)
```
Max of $1$ happens when using prefix $[-1, 0, 2]$

Solving MCSSE is similar, except "backwards":

**MCSS ending at position $i$:**


Here, $v$ is the sum of $a[0\cdots j]$. The key observation here is that for a location $i$, if we subtract prefix sum up to position $i$ from $v$ then we have the subsequence sum for $a[i\cdots j]$. The first `scan` computes these prefixes and the `reduce` identifies which difference is maximum. 


<img width=70% src="figures\mcsse_fig.jpg"/>





### Sparc Code

<p><span class="math display">\[\begin{array}{l}  
\mathit{MCSSEOpt}~a~j=  
\\  
~~~~\texttt{let}  
\\  
~~~~~~~~(b,v) = \mathit{scan}~\text{'}\,{\texttt{+}}\,\text{'}~0~a[0 \cdots j]  
\\  
~~~~~~~~w= \mathit{reduce}~\mathit{min}~\infty~b  
\\  
~~~~\texttt{in}  
\\  
~~~~~~~~v - w   
\\  
~~~~\texttt{end}  
\end{array}\]</span></p>

<!-- ![mcsse_fig.jpg](mcsse_fig.jpg) -->

### Python Code

```python
>>> b, v = scan(add, 0, [1,-2,0,3])
>>> print('b=', b, 'v=', v)
b= [1, -1, -1, 2] v= 2

>>> w = reduce(min, -math.inf, b)
>>> print('w=', w)
w= -1

>>> print('v-w=', v-w)
v-w= 3
```

Max of $3$ happens for subsequence $[0,3]$

## Any ideas to improve the work? 


<img width=70% src="figures\mcssps.jpeg"/>



<!-- ![mcssps.jpeg](mcssps.jpeg) -->




We need to return a total of four values:

- the max subsequence sum, $m$

- the max prefix sum, $p$

- the max suffix sum, $s$ and

- the overall sum $t$. 
> [<span style="color:red">**Question**</span>: Why do we need this overall sum?]



Having this information from the subproblems is enough to produce a similar answer tuple for all levels up, in constant work and span per level. Thus what we have discovered is that to solve the strengthened problem efficiently we have to strengthen the problem once again. 

Thus if the recursive calls return ($𝑚_1,𝑝_1,𝑠_1,𝑡_1$)
 and ($𝑚_2,𝑝_2,𝑠_2,𝑡_2$),
 
then we combine the recursive calls and obtain

> (max($𝑠_1+𝑝_2$,$𝑚_1$,$𝑚_2$), max($𝑝_1,𝑡_1+𝑝_2$),max($𝑠_1+𝑡_2,𝑠_2$),$𝑡_1+𝑡_2$).


<img width=70% src="figures\mcsssp_sol.jpeg"/>
